In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numba
import grid2op
import numpy as np
from lightsim2grid import LightSimBackend          # highly recommended!
from MyRewards import RLReward, MarginReward, RedisActionCost

In [ ]:
excluded_numbers = {} # "31500_"

# Create a list of strings with increments of 1000 or 500, excluding the ones in the set
number_list = []

start, end = 35000, 67000

for number in range(start, end + 1):
    increment = 1000 if number <= 29000 else 1000
    if number % increment == 0:
        number_str = str(number) + "_"
        if number_str not in excluded_numbers:
            number_list.append(number_str)

print(number_list)

In [ ]:
for string_number in number_list:
    
    NB_EPISODE = 32
    # choose the desired model
    lambdaa = 0.0
    line_switch_cost = 1.5

    model = string_number
    signature = "36bus_posttrain-model_anmol_LODF-try12" + "_" + str(line_switch_cost) + "_" + str(lambdaa)
    inference_process = "inf1_"
    fileame = "saved_models" + "_" + str(line_switch_cost) + "_" + str(lambdaa) + ".txt"
    
    if inference_process=="inf1_":
        from agent_exp import MyAgent
    else:
        from agent_exp2 import MyAgent

    def make_agent(env, submission_dir):
        """
        This function will be used by codalab to create your agent. It should accept exactly an environment and a path
        to your submission directory and return a valid agent.
        """

        def powerFlowLimitsFunction(env):
            """
            Power Flow Limits array (necessary, needed only once at the start) 
            Size: number of branches in the "initial/starting" grid.
            power-flow limits on each line in the "INITIAL" power-grid. Go to array to extract the power-line limit.
            after my thorough research, power flow limits on each grid2op branch (transmission line and transformer) can be found as:
            """
            baseVoltagesForEachLine = np.array([138]*45 + [345]*3 + [138]*7 + [345]*4)         # manually building it since the base voltages can be recognized from the previous section.
            powerFlowLimits = np.abs(env.get_thermal_limit())*baseVoltagesForEachLine*1000*np.sqrt(3)
            powerFlowLimits = powerFlowLimits/1e6                          # power flow limits in MWs
            return powerFlowLimits

        powerFlowLimits = powerFlowLimitsFunction(env) # will not change with time steps
        init_obs = env.reset()

        agent = MyAgent(env.observation_space, env.action_space, env._init_grid_path, init_obs, powerFlowLimits)
        import os
        agent.load(os.path.join(submission_dir, model + signature, "DDDQN_random_train.h5"))
        return agent

    # reward must be a subclass of grid2op.Reward.BaseReward.BaseReward:
    selected_reward_function = RLReward(lambdaa, line_switch_cost)
    other_rewards = { "margin_reward": MarginReward, "action_cost": RedisActionCost }


    env_name = "l2rpn_neurips_2020_track1_small_test_random_32"
    #"l2rpn_neurips_2020_track1_small_December" # "l2rpn_neurips_2020_track1_small_test_random_28"

    env = grid2op.make(env_name,
                       reward_class=selected_reward_function,
                       #param=custom_params,
                       other_rewards = other_rewards,
                       backend=LightSimBackend(),
                       opponent_init_budget=0, opponent_budget_per_ts=0
                       )


    agent = make_agent(env, "C:\\Users\\dwivea2\\Desktop\\RL code\\CustomAgent _LODF")


    from grid2op.Runner import Runner
    from tqdm.notebook import tqdm
    runner = Runner(**env.get_params_for_runner(), agentClass=None, agentInstance=agent)


    PATH_SAVE = inference_process + model + "36bus_saved_experiment_anmol" + "_" + str(line_switch_cost) + "_" + str(lambdaa)
    res = runner.run(nb_episode=NB_EPISODE, path_save=PATH_SAVE, env_seeds=list(range(NB_EPISODE)))

    avgSurvTime = []
    avgCumRewad = []

    for chron_name, _, cum_reward, nb_time_step, max_ts in res:
        avgSurvTime.append(nb_time_step)
        avgCumRewad.append(cum_reward)

    #msg_tmp = "\tFor chronics located at {}\n".format(chron_name)
    #msg_tmp += "\t\t - cumulative reward: {:.2f}\n".format(cum_reward)
    #msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}\n".format(nb_time_step, max_ts)

    
    with open(fileame, "a") as result_file:
        result_file.write(f"Model: {string_number}\n")
        
        result_file.write("Average Survival Time:\n")
        result_file.write(", ".join(map(str, avgSurvTime)) + "\n")

    avg_survival_time = np.average(avgSurvTime)
    max_timestep = env.chronics_handler.max_timestep()
    avg_cumulative_reward = np.average(avgCumRewad)

    # Append the averages to the result file
    with open(fileame, "a") as result_file:
        result_file.write(f"Average Survival Time: {avg_survival_time} / {max_timestep} time steps.\n")
        result_file.write(f"Average Cumulative Reward is: {avg_cumulative_reward}\n")
        result_file.write("\n")

    print(avgSurvTime)
    print("Average Survival Time:", avg_survival_time, "/", max_timestep, "time steps.")
    print("Average Cumulative Reward is:", avg_cumulative_reward)